Importamos las librerías

In [12]:
from configuration.invest_config import invest, datos_extraidos
from configuration.codigo_graficos import grafico_valor,grafico_suma_ibex35, grafico_descenso_por_valor, grafico_descenso_por_valor_categoria
import numpy as np
import pandas as pd
import datetime
import os
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display
import matplotlib.pyplot as plt
from plotly.offline import iplot
from varname import nameof

Cargamos todos los datos descargados y le asignamos nombre a las variables

In [13]:
carga_datos=datos_extraidos.datos_extraidos()
valores=carga_datos[0]
valores_name=carga_datos[1]
dic_valores=carga_datos[2]

In [14]:
#Extramos precios de apertura
dict_open={}
n=1
for i in valores:
    dict_open[n]=i["Open"]
    n+=1

Creamos un gráfico de resumen de cualquier valore del IBEX35

In [15]:
grafico_valor("GRF", dic_valores)

Creamos una gráfica con la evolución del precio total (suma) de los valores del IBEX35

In [16]:
#Sacamos la suma de evolución de todos los valores del IBEX35
df_suma_total=pd.DataFrame(dict_open).sum(axis=1)
df_suma_IBEX35=pd.concat([dic_valores["BBVA"]["Date"], df_suma_total], axis=1)
df_suma_IBEX35=df_suma_IBEX35.rename(columns={0:"Open"})

In [17]:
grafico_suma_ibex35(df_suma_IBEX35)

Encontramos el pico más alto y bajo del IBEX35 al declararse el confinamiento

In [18]:
df_suma_IBEX35_2022=df_suma_IBEX35[df_suma_IBEX35["Date"].str.startswith("2020")]
max_IBEX35_COVID=df_suma_IBEX35_2022[df_suma_IBEX35_2022["Open"]==df_suma_IBEX35_2022["Open"].max()]
min_IBEX35_COVID=df_suma_IBEX35_2022[df_suma_IBEX35_2022["Open"]==df_suma_IBEX35_2022["Open"].min()]

print("Pico máximo:")
print(max_IBEX35_COVID)
print("Pico mínimo:")
print(min_IBEX35_COVID)

Pico máximo:
           Date        Open
111  2020-02-17  745.290393
Pico mínimo:
           Date        Open
115  2020-03-16  479.870191


Encontramos la media en % en la que descendieron los valores en la caida pico

In [ ]:
print("Descenso en % del IBEX35=",100-(479.870191*100/745.290393))

Descenso en % del IBEX35= 35.61299119013333


Calculamos cuánto descendió el valor de cada valor en % en la caida pico del IBEX35

In [ ]:
dif_valores_pico={}
for valor in dic_valores:
    try:
        max_value=float(dic_valores[valor]["Open"][dic_valores[valor].index==111].head(1))
        min_value=float(dic_valores[valor]["Open"][dic_valores[valor].index==115].head(1))
        dif=100-(min_value*100/max_value)
        dif_valores_pico[valor]=dif
    except:
        pass
df_descenso_empresa = pd.DataFrame(dif_valores_pico.items(), columns=['Empresa', 'Valor'])

In [ ]:
grafico_descenso_por_valor(df_descenso_empresa)

In [ ]:
def grafico_descenso_por_valor1(df_descenso_empresa):
    # Devuelve un grafico de barras con el descenso del valor por empresa el el momento pico COVID
    # Crear gráfico de barras
    fig = px.bar(df_descenso_empresa, x=df_descenso_empresa["Empresa"], y='Valor')

    # Calcular la media de los valores de la columna 'Valor'
    mean_value = df_descenso_empresa['Valor'].mean()

    # Agregar una línea horizontal en la media de la columna 'Valor'
    fig.add_shape(type="line",
                  x0=-0.5,
                  y0=mean_value,
                  x1=len(df_descenso_empresa["Empresa"])-0.5,
                  y1=mean_value,
                  line=dict(color="red", width=2),
                  name="Media") # Agregar la opción 'name' para que aparezca en la leyenda

    # Personalizar el diseño del gráfico
    fig.update_layout(title = dict(text='Descenso por valor en la caída pico de COVID', x=0.5,
                            font=dict(size=24)),
                      autosize=False,
                      width=1000,
                      height=600,
                      paper_bgcolor='white',
                      plot_bgcolor='white', 
                      font=dict(size=16),
                      showlegend=True, # Activar la leyenda
                      xaxis_title="Índice",
                      yaxis_title="Descenso 17/02/2020-16/03/2020 (%)")

    # Mostrar el gráfico
    return (fig)


In [ ]:
grafico_descenso_por_valor1(df_descenso_empresa)

Editamos la gráfica anterior para separar los valores por sector

In [ ]:
#Diccionario clasificacion valores por sector
sectores={"Banca":["BBVA","BKT","CABK","SAN","SAB", "UNI"],
          "Software":["CLNX","TEF","AMS","LOG"],
          "Energia":["IBE","NTGY","RED","ENG","ELE","ANE","ANA"],
          "Movilidad":["AENA","IAG","FER"],
          "Industrial":["MTS","ACX","ACS"],
          "Otros":["MAP","MEL","ITX","GRF","FDR","COL"],
          }
sectores_invertido = {valor: clave for clave, valores in sectores.items() for valor in valores}

In [ ]:
lista_categoria=[]
for empresa in df_descenso_empresa["Empresa"]:
    lista_categoria.append(sectores_invertido[empresa])
df_descenso_empresa["categoria"]=lista_categoria

In [ ]:
grafico_descenso_por_valor_categoria(df_descenso_empresa)

Buscamos en qué porcentaje de caída tuvo influencia cada sector

In [ ]:
descenso_IBEX_35_pico=745.290393-479.870191
descenso_por_valor_pico={}

for valor in dic_valores:
    try:
        descenso_por_valor_pico[valor]=100-(dic_valores[valor]["Open"][115]*100)/dic_valores[valor]["Open"][111]
    except:
        pass

df_descenso_porcentaje = pd.DataFrame(descenso_por_valor_pico.items(), columns=['Empresa', 'Valor'])

ls_categoria=[]
for empresa in df_descenso_porcentaje["Empresa"]:
    ls_categoria.append(sectores_invertido[empresa])

df_descenso_porcentaje["Categoria"]=ls_categoria
df_descenso_categoria=df_descenso_porcentaje.groupby("Categoria").mean()
ls_porcentaje_descenso_categoria=[]
valor_total_descenso=sum(df_descenso_categoria["Valor"])
for valor in df_descenso_categoria["Valor"]:
    ls_porcentaje_descenso_categoria.append(valor*100/valor_total_descenso)
df_descenso_categoria["descenso_porcentual"]=ls_porcentaje_descenso_categoria

In [ ]:
# Crear el gráfico circular con Plotly
fig = go.Figure(data=[go.Pie(labels=df_descenso_categoria.index, values=df_descenso_categoria['descenso_porcentual'])])

fig.update_layout(
    title={
        'text': "Distribución del descenso por sector",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    width=600,
    height=500,
)
# Mostrar el gráfico
fig.show()

Descenso medio por categoría

In [ ]:
media_descenso_categoría=df_descenso_empresa.copy()
media_descenso_categoría.groupby("categoria").mean()

,Valor
categoria,
Banca,41.155189
Energia,30.747472
Industrial,45.057266
Movilidad,48.291905
Otros,36.402413
Software,34.176983


In [ ]:
def grafico_evolucion_volumen(valor, dic_valores):
  #Gráfico de evolución del volumen
  caida = dic_valores[valor]
  # MEDIA DEL PRECIO DE APERTURA DEL IBEX35
  trace1 = go.Scatter(
                      x = caida['Date'],
                      y = caida['Volume'],
                      name = 'citations',
                      mode= 'lines',
                      marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                      )

  data = [trace1]

  layout = dict(title = dict(text=f'VOLUMEN {valor}', x=0.5),
              xaxis= dict(title= 'FECHA',ticklen= 5),
              yaxis= dict(title= 'VOLUMEN',ticklen= 5)
            )

  fig = go.Figure(data = data, layout=layout)
  return(fig)

In [ ]:
for valor in dic_valores:
    iplot(grafico_evolucion_volumen(valor, dic_valores))

In [ ]:
def grafico_evolucion_volumen_todos(dic_valores):
  #Gráfico de evolución del volumen
  caida = dic_valores[valor]
  # MEDIA DEL PRECIO DE APERTURA DEL IBEX35
  trace1 = go.Scatter(
                      x = caida['Date'],
                      y = caida['Volume'],
                      name = 'citations',
                      mode= 'lines',
                      marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                      )

  data = [trace1]

  layout = dict(title = dict(text=f'VOLUMEN {valor}', x=0.5),
              xaxis= dict(title= 'FECHA',ticklen= 5),
              yaxis= dict(title= 'VOLUMEN',ticklen= 5)
            )

  fig = go.Figure(data = data, layout=layout)
  return(fig)

In [ ]:
trazos_empresas = []
for nombre, datos in dic_valores.items():
    trazo = go.Scatter(
        x=datos['Date'],
        y=datos['Volume'],
        name=nombre,
        line=dict(width=2),
        hovertemplate='%{y:.2f} unidades<br>%{x|%d-%m-%Y}<extra></extra>'
    )
    trazos_empresas.append(trazo)
layout = go.Layout(
    title='Evolución del volumen de empresas',
    xaxis=dict(title='Fecha'),
    yaxis=dict(title='Volumen'),
    legend=dict(orientation='h', y=1.05)
)
fig = go.Figure(data=trazos_empresas, layout=layout)
iplot(fig)

Aumento en % del IBEX35 desde el final de la caida hasta la actualidad

In [ ]:
abs(100-(float(df_suma_IBEX35.tail(1)["Open"])*100/float(df_suma_IBEX35[df_suma_IBEX35.index==115]["Open"])))

54.32215105447813

Variacion en % entre antes de la caida y ahora mismo

In [ ]:
abs(100-(float(df_suma_IBEX35.tail(1)["Open"])*100/float(df_suma_IBEX35[df_suma_IBEX35.index==111]["Open"])))

0.6365831244000333

Variacion inicio caida y actual por empresa

In [22]:
variacion_inicio_actual={}

for empresa in dic_valores:
    try:
        caida_actual=100-(dic_valores[empresa]["Open"][275]*100/dic_valores[empresa]["Open"][111])
        caida_actual=round(caida_actual*(-1),2)
        variacion_inicio_actual[empresa]=caida_actual
    except:
        pass

In [30]:
valores = list(variacion_inicio_actual.values())
empresas = list(variacion_inicio_actual.keys())
trace = go.Bar(x=empresas, y=valores)
layout = go.Layout(title='Variación de cada valor (Actual respecto a Pre-Covid)')
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [29]:
labels = ['Superiores a Pre-Covid', 'Inferiores a Pre-Covid']
values = [35.71, 64.29]
trace = go.Pie(labels=labels, values=values)
layout = go.Layout(title='Distribución de empresas según su valor actual respecto a su situación Pre-Covid')
fig = go.Figure(data=[trace], layout=layout)
fig.show()